# Allgemein

In [34]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [35]:
import graphviz as gv
import heapq

In [36]:
def to_list(State): 
    return [list(row) for row in State]

In [37]:
def to_tuple(State):
    return tuple(tuple(row) for row in State)

# Suchprobleme

## Breitensuche (Breadth First Search)

In [38]:
def rec_path_to(state, Parent):
    p = Parent[state]
    if p == state:
        return [state]
    return rec_path_to(p, Parent) + [state]

In [39]:
def bf_search(start, goal, next_states):
    Frontier = { start }
    Visited  = set()
    Parent   = { start: start }
    while Frontier:
        NewFrontier = set()
        for s in Frontier:
            for ns in next_states(s):
                if ns not in Visited and ns not in Frontier:
                    NewFrontier.add(ns)
                    Parent[ns] = s
                    if ns == goal:
                        print("number of states: ", len(Visited) + len(Frontier) + len(NewFrontier))
                        return rec_path_to(goal, Parent)
        Visited |= Frontier
        Frontier = NewFrontier

## Tiefensuche mit Stack (Depth First Search)

In [40]:
def iter_path_to(state, Parent):
    Path = [state]
    while state != Parent[state]:
        state = Parent[state]
        Path  = [ state ] + Path
    return Path

In [41]:
def df_search(start, goal, next_states):
    Stack  = [start]
    Parent = { start: start }
    while Stack:
        state = Stack.pop()
        for ns in next_states(state):            
            if ns not in Parent:
                Parent[ns] = state
                Stack.append(ns)
                if ns == goal:
                    return iter_path_to(goal, Parent)

## Iterative Tiefensuche (Iterative Deepening)

In [42]:
def depth_limited_search(state, goal, next_states, Path, PathSet, limit):
    if state == goal:
        return Path
    if len(Path) == limit:
        return None
    for ns in next_states(state):
        if ns not in PathSet:
            Path   .append(ns)
            PathSet.add(ns)
            Result = depth_limited_search(ns, goal, next_states, Path, PathSet, limit)
            if Result:
                return Result
            Path   .pop()
            PathSet.remove(ns) # remove this line for faster, but non-optimal solution
    return None

In [43]:
def id_search(start, goal, next_states):
    limit = 32
    while True:
        Path = depth_limited_search(start, goal, next_states, [start], { start }, limit)
        if Path is not None:
            return Path
        limit += 1
        print(f'limit = {limit}')

## Best First Search

In [44]:
def bfs_search(start, goal, next_states, heuristic):
    PrioQueue = [ (heuristic(start, goal), [start]) ]
    while PrioQueue:
        _, Path = heapq.heappop(PrioQueue)
        state   = Path[-1]
        if state == goal:
            return Path
        for ns in next_states(state):
            if ns not in Path:
                d = heuristic(ns, goal)
                heapq.heappush(PrioQueue, (d, Path + [ns]))

## Beispiel Missionare

$\texttt{problem}(m, i)$ is `True` if there is a problem on a shore that has $m$ missionaries and $i$ infidels.
For a problem to arise, the number $m$ of missionaries needs to be greater than $0$ but less than the number $i$ of
infidels.

In [45]:
def problem(m, i): 
    return 0 < m < i

def no_problem(m, i): 
    return not problem(m, i) and not problem(3 - m, 3 - i)

In [46]:
def next_states_mis(state):
    m, i, b = state
    if  b == 1:
        return { (m-mb, i-ib, 0) for mb in range(m+1)
                                 for ib in range(i+1)
                                 if 1 <= mb + ib <= 2 and no_problem(m-mb, i-ib) 
               }
    else:
        return { (m+mb, i+ib, 1) for mb in range(3-m+1)
                                 for ib in range(3-i+1)
                                 if 1 <= mb + ib <= 2 and no_problem(m+mb, i+ib) 
               }

Initially, all missionaries, all infidels and the boat are on the left shore.
The goal is to have everybody on the right shore, hence the numbers on the left shore
should all be $0$.

In [47]:
start = (3, 3, 1)
goal  = (0, 0, 0)

In [48]:
def fillCharsLeft(x, n):
    s = str(x)
    m = n - len(s)
    return m * " " + s

def fillCharsRight(x, n):
    s = str(x)
    m = n - len(s)
    return s + m * " "

def fillCharsBoth(x, n):
    s  = str(x)
    ml = (n     - len(s)) // 2
    mr = (n + 1 - len(s)) // 2
    return ml * " " + s + mr * " "

def printState(m, k, b):
     print( fillCharsRight(m * "M", 6) + 
            fillCharsRight(k * "K", 6) + 
            fillCharsRight(b * "B", 3) + "    |~~~~~|    " + 
            fillCharsLeft((3 - m) * "M", 6) + 
            fillCharsLeft((3 - k) * "K", 6) + 
            fillCharsLeft((1 - b) * "B", 3) 
          )

def printBoat(m1, k1, b1, m2, k2, b2):
    if b1 == 1:
        if m1 < m2:
            print("Error in printBoat: negative number of missionaries in the boat!")
            return
        if k1 < k2:
            print("Error in printBoat: negative number of infidels in the boat!")
            return
        print(19*" " + "> " + fillCharsBoth((m1-m2)*"M" + " " + (k1-k2)*"K", 3) + " >")
    else:
        if m1 > m2:
            print("Error in printBoat: negative number of missionaries in the boat!")
            return
        if k1 > k2:
            print("Error in printBoat: negative number of infidels in the boat!")
            return
        print(19*" " + "< " + fillCharsBoth((m2-m1)*"M" + " " + (k2-k1)*"K", 3) + " <")

def printPath(Path):
    print("Solution:\n")
    for i in range(len(Path) - 1):
        m1, k1, b1 = Path[i]
        m2, k2, b2 = Path[i+1]
        printState(m1, k1, b1)
        printBoat(m1, k1, b1, m2, k2, b2)
    m, k, b = Path[-1]
    printState(m, k, b)

In [49]:
%%time
%memit Path = bf_search(start, goal, next_states_mis)

number of states:  15
peak memory: 93.36 MiB, increment: 0.00 MiB
CPU times: total: 46.9 ms
Wall time: 1 s


In [50]:
%%time
%memit Path = df_search(start, goal, next_states_mis)

peak memory: 93.36 MiB, increment: 0.00 MiB
CPU times: total: 46.9 ms
Wall time: 985 ms


In [51]:
%%time
%memit Path = id_search(start, goal, next_states_mis)

peak memory: 93.36 MiB, increment: 0.00 MiB
CPU times: total: 46.9 ms
Wall time: 989 ms


## Beispiel Sliding Puzzle

### Animation

The package `ipycanvas`, which is imported below, can be installed using the following command:
```
    conda install -c conda-forge ipycanvas
```
This package is useful for drawings and animations.  Its documentation can be found at:
  https://ipycanvas.readthedocs.io/en/latest/.

In [52]:
import ipycanvas as cnv

The module `time` is part of the standard library, so it is preinstalled.  We have imported it because we need the function `time.sleep(secs)` to pause the animation for a specified time.

In [53]:
import time

The global variable `Colors` specifies the colors of the tiles.

In [54]:
Colors = ['white', 'lightblue', 'pink', 'magenta', 'orange', 'red', 'yellow', 'lightgreen', 'gold',
          'CornFlowerBlue', 'Coral', 'Cyan', 'orchid', 'DarkSalmon', 'DeepPink', 'green'
         ] 

The global variable `size` specifies the size of one tile in pixels.

In [55]:
size = 100

The function `draw(State, canvas, dx, dy, tile, x)` draws a given `State` of the sliding puzzle, where `tile` has been moved by `offset` pixels into the direction `(dx, dy)`.

In [56]:
def draw(State, canvas, dx, dy, tile, offset):
    canvas.text_align    = 'center'
    canvas.text_baseline = 'middle'
    with cnv.hold_canvas(canvas):
        canvas.clear()
        n = len(State)
        for row in range(n):
            for col in range(n):
                tile_to_draw = State[row][col]
                color = Colors[tile_to_draw]
                canvas.fill_style = color
                if tile_to_draw not in (0, tile):
                    x = col * size
                    y = row * size
                    canvas.fill_rect(x, y, size, size)
                    canvas.line_width = 3.0
                    x += size // 2
                    y += size // 2
                    canvas.stroke_text(str(tile_to_draw), x, y)
                elif tile_to_draw == tile:
                    x = col * size + offset * dx
                    y = row * size + offset * dy
                    canvas.fill_rect(x, y, size, size)
                    canvas.line_width = 3.0
                    x += size // 2
                    y += size // 2
                    if tile_to_draw != 0:
                        canvas.stroke_text(str(tile_to_draw), x, y)

In [57]:
def create_canvas(n): 
    canvas = cnv.Canvas(size=(size * n, size * n))
    canvas.font = '100px serif'
    return canvas

The global variable `delay` controls the speed of the animation.

In [58]:
delay = 0.0005

The function call `tile_and_direction(state, next_state)` takes a state and the state that follows this state and returns a triple `(tile, dx, dy)` where `tile` is the tile that is moved to transform `state` into `next_state` and `(dx, dy)` is the direction in which this tile is moved.

In [59]:
def tile_and_direction(state, next_state):
    row0, col0 = find_tile(0, state)
    row1, col1 = find_tile(0, next_state)
    return state[row1][col1], col0-col1, row0-row1

Given a list of states representing a solution to the sliding puzzle, the function call 
`animation(Solution)` animates the solution.

In [60]:
def animation(Solution):
    start = Solution[0]
    n = len(start)
    canvas = create_canvas(n)
    draw(start, canvas, 0, 0, 0, 0)
    m = len(Solution)
    display(canvas)
    for i in range(m-1):
        state = Solution[i]
        tile, dx, dy = tile_and_direction(state, Solution[i+1])
        for offset in range(size+1):
            draw(state, canvas, dx, dy, tile, offset)
            time.sleep(delay)

### Code

In [61]:
def move_dir(State, row, col, dx, dy):
    State = to_list(State)
    State[row     ][col     ] = State[row + dx][col + dy]
    State[row + dx][col + dy] = 0
    return to_tuple(State)

In [62]:
def find_tile(tile, State):
    n = len(State)
    for row in range(n):
        for col in range(n):
            if State[row][col] == tile:
                return row, col

In [63]:
def next_states_sliding(State):
    n          = len(State)
    row, col   = find_tile(0, State)
    New_States = set()
    Directions = [ (1, 0), (-1, 0), (0, 1), (0, -1) ]
    for dx, dy in Directions:
        if row + dx in range(n) and col + dy in range(n):
            New_States.add(move_dir(State, row, col, dx, dy))
    return New_States

In [64]:
start = ( (8, 0, 6),
          (5, 4, 7),
          (2, 3, 1)
        )
goal = ( (0, 1, 2), 
         (3, 4, 5), 
         (6, 7, 8)
       )

In [65]:
def manhattan(stateA, stateB):
    n = len(stateA)
    PositionsB = {}
    for row in range(n):
        for col in range(n): 
            tile = stateB[row][col]
            PositionsB[tile] = (row, col)
    result = 0
    for rowA in range(n):
        for colA in range(n): 
            tile = stateA[rowA][colA]
            if tile != 0:
                rowB, colB = PositionsB[tile]
                result += abs(rowA - rowB) + abs(colA - colB)
    return result

In [66]:
%%time
%memit Path = bf_search(start, goal, next_states_sliding)

number of states:  181440
peak memory: 156.66 MiB, increment: 63.35 MiB
CPU times: total: 2.25 s
Wall time: 3.12 s


In [67]:
%%time
%memit Path = df_search(start, goal, next_states_sliding)

peak memory: 111.88 MiB, increment: 3.48 MiB
CPU times: total: 1.02 s
Wall time: 1.85 s


In [68]:
%%time
%memit Path = id_search(start, goal, next_states_sliding)

peak memory: 110.62 MiB, increment: 0.00 MiB
CPU times: total: 5.06 s
Wall time: 5.94 s


In [69]:
%%time
%memit bfs_search(start, goal, next_states_sliding, manhattan)

peak memory: 107.63 MiB, increment: 0.01 MiB
CPU times: total: 62.5 ms
Wall time: 898 ms


In [70]:
#animation(Path)